[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hak9238/hak9238/blob/main/koalpaca-polyglot-12.8b.ipynb)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "beomi/KoAlpaca-Polyglot-12.8B"
offload_folder = "/Users/hakhyunlee/offload"  # 쓰기 가능한 경로로 설정

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# FP16으로 모델 로드하고, device_map을 auto로 설정하여 자원을 자동으로 분배
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",  # GPU/CPU 자원을 자동으로 분배
    torch_dtype="float16",  # FP16 사용으로 메모리 절약
    offload_folder=offload_folder  # 메모리 부족 시 디스크로 일부 오프로드
)

print("✅ KoAlpaca-Polyglot-12.8B 모델이 성공적으로 로드되었습니다!")


/opt/anaconda3/lib/python3.12/site-packages/accelerate/utils/modeling.py:1536: UserWarning: Current model requires 18876808.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.


✅ KoAlpaca-Polyglot-12.8B 모델이 성공적으로 로드되었습니다!


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 크롤링할 날짜 범위 설정 (2025년 1월 1일부터 1월 3일까지)
start_date = '20250101'
end_date = '20250103'
date_range = pd.date_range(start=start_date, end=end_date)

# 네이버 뉴스 카테고리 코드 설정
categories = {
    '정치': '100',
    '경제': '101',
    '사회': '102',
    '생활/문화': '103',
    '세계': '104',
    'IT/과학': '105',
    '오피니언': '110',
    '포토': '115',
    'TV': '120'
}
base_url = 'https://news.naver.com/main/list.naver'

# HTTP 요청 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/58.0.3029.110 Safari/537.3'
}

# 전체 뉴스 데이터를 저장할 리스트 초기화
all_news = []

# 각 날짜와 카테고리에 대해 크롤링 수행
for single_date in date_range:
    date_str = single_date.strftime('%Y%m%d')
    print(f"📅 {date_str} 날짜의 기사를 수집합니다...")

    for category_name, category_code in categories.items():
        print(f"🗂️ '{category_name}' 카테고리의 기사를 수집합니다...")

        titles = []
        contents = []
        links_set = set()  # 중복 방지를 위한 집합

        # 페이지 번호를 1부터 3까지 순회
        for page in range(1, 4):
            # 현재 페이지의 URL 생성
            params = {
                'mode': 'LS2D',
                'mid': 'shm',
                'sid1': category_code,
                'date': date_str,
                'page': page
            }
            response = requests.get(base_url, headers=headers, params=params)
            soup = BeautifulSoup(response.text, 'html.parser')

            # 현재 페이지의 뉴스 기사 링크 추출
            articles = soup.select('.newsflash_body .type06_headline li dl dt a')
            links = [article['href'] for article in articles]

            # 각 기사에 접근하여 제목과 본문 추출
            for link in links:
                if link not in links_set:
                    links_set.add(link)
                    article_response = requests.get(link, headers=headers)
                    article_soup = BeautifulSoup(article_response.text, 'html.parser')

                    # 제목 추출
                    title_element = article_soup.find('h2', {'id': 'title_area'})
                    if title_element:
                        title = title_element.get_text(strip=True)
                    else:
                        title = '제목을 찾을 수 없습니다.'

                    # 본문 추출
                    content_element = article_soup.find('div', {'id': 'newsct_article'})
                    if content_element:
                        content = content_element.get_text(strip=True)
                    else:
                        content = '본문을 찾을 수 없습니다.'

                    titles.append(title)
                    contents.append(content)

        # 데이터프레임 생성 및 중복 제거
        news_df = pd.DataFrame({
            'date': date_str,
            'category': category_name,
            'title': titles,
            'content': contents
        })
        news_df.drop_duplicates(subset=['title'], inplace=True)

        # 수집한 데이터를 리스트에 추가
        all_news.append(news_df)

# 모든 날짜와 카테고리의 데이터를 하나의 데이터프레임으로 결합
final_news_df = pd.concat(all_news, ignore_index=True)

# 저장할 폴더 경로 설정
save_folder = '/Users/hakhyunlee/Desktop/대학원/실습 예제/뉴스 파일'

# 폴더가 존재하지 않으면 생성
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# 파일명 설정
file_name = f'뉴스_{start_date}_to_{end_date}.csv'
save_path = os.path.join(save_folder, file_name)

# CSV 파일로 저장
final_news_df.to_csv(save_path, index=False, encoding='utf-8-sig')

print(f"📰 뉴스 데이터가 '{save_path}'에 성공적으로 저장되었습니다.")





📅 20250101 날짜의 기사를 수집합니다...
🗂️ '정치' 카테고리의 기사를 수집합니다...
🗂️ '경제' 카테고리의 기사를 수집합니다...
🗂️ '사회' 카테고리의 기사를 수집합니다...
🗂️ '생활/문화' 카테고리의 기사를 수집합니다...
🗂️ '세계' 카테고리의 기사를 수집합니다...
🗂️ 'IT/과학' 카테고리의 기사를 수집합니다...
🗂️ '오피니언' 카테고리의 기사를 수집합니다...
🗂️ '포토' 카테고리의 기사를 수집합니다...
🗂️ 'TV' 카테고리의 기사를 수집합니다...
📅 20250102 날짜의 기사를 수집합니다...
🗂️ '정치' 카테고리의 기사를 수집합니다...
🗂️ '경제' 카테고리의 기사를 수집합니다...
🗂️ '사회' 카테고리의 기사를 수집합니다...
🗂️ '생활/문화' 카테고리의 기사를 수집합니다...
🗂️ '세계' 카테고리의 기사를 수집합니다...
🗂️ 'IT/과학' 카테고리의 기사를 수집합니다...
🗂️ '오피니언' 카테고리의 기사를 수집합니다...
🗂️ '포토' 카테고리의 기사를 수집합니다...
🗂️ 'TV' 카테고리의 기사를 수집합니다...
📅 20250103 날짜의 기사를 수집합니다...
🗂️ '정치' 카테고리의 기사를 수집합니다...
🗂️ '경제' 카테고리의 기사를 수집합니다...
🗂️ '사회' 카테고리의 기사를 수집합니다...
🗂️ '생활/문화' 카테고리의 기사를 수집합니다...
🗂️ '세계' 카테고리의 기사를 수집합니다...
🗂️ 'IT/과학' 카테고리의 기사를 수집합니다...
🗂️ '오피니언' 카테고리의 기사를 수집합니다...
🗂️ '포토' 카테고리의 기사를 수집합니다...
🗂️ 'TV' 카테고리의 기사를 수집합니다...
📰 뉴스 데이터가 '/Users/hakhyunlee/Desktop/대학원/실습 예제/뉴스 파일/뉴스_20250101_to_20250103.csv'에 성공적으로 저장되었습니다.
